### Strategy (solve_wordle)
- Initial Conditions:
  - Set puzzle_word
  - X = Start with universe of 5 letter words along with freqency of usage.
  - Set current_universe = X
- Start
  - Pick guess (pick the most frequent word in current_universe that we haven't picked previously)
  - If guess == puzzle_word)
    - Goto End
  - Get wordle info about how close guess is to the correct word:
    - wordle_info = create_wordle_info(<guess>, <puzzle_word>)
      - Example wordle_info, create_wordle_info("exact", "crane") = ( [('a', 3)], Dict('x' => (0, 0), 'c' => (1, 1), 'e' => (1, 1), 't' => (0, 0)) )
  - Use this match info to filter existing universe.
    - current_universe = filter_universe(wordle_info, current_universe)
  - Goto Start
- End
    - Return guess 

### Environment
- We assume that the server for this notebook has been launched at the top level for this project
  as below we look for the Wordle database in the directory:  ENV["PWD"] * "/data/" .
- The Wordle database is a CSV file with schema: word, freq. Here.
    - The values in the word column are strings assumed to match: r"[a-z]{N}", where N is a integer -- N is the same value for all rows.
    - The items in the freq column are strings that represent floating point numbers. 

### Use/Import Packages

In [ ]:
using DataFrames
using StatsBase
using Statistics
using CSV
import Wordle
using Wordle: create_wordle_info, filter_universe, freq_letter_strat, solve_wordle

### Functions to Solve Wordle Puzzle

### Read in Wordle Words
- Five letter words and their frequencies are stored in two files.
- Read them in and create a DataFrame, then sort the words from most to least used.
- Words and frequencies obtained from the Mathematica function WordFrequencyData.

### Setup Wordle Database

In [ ]:
## Change this path to point to the wordle_db.csv path.
## Load the Wordle database as a Dataframe with schema: word(5 letter lowercase word as String),freq(Relative frequency of word as Float64)
const WORDLE_DF =  DataFrame(CSV.File(joinpath(dirname(pathof(Wordle)), "../data",  "wordle_db.csv"); header=6, types=[String7, Float64], comment="#"));

### Testing

In [ ]:
names(WORDLE_DF)

In [ ]:
pathof(Wordle)

In [ ]:
create_wordle_info("exact", "crane")

In [ ]:
(info, d) = create_wordle_info("there", "their")
println(info)
println(d)

In [ ]:
filter_universe((info, d), WORDLE_DF[!, :word])

In [ ]:
uni = filter(:word => x -> x in ["where", "state", "other", "child", "there", "their", "about"], WORDLE_DF)
solve_wordle("other", uni)

In [ ]:
create_wordle_info("other", "child")

In [ ]:
solve_wordle("child", uni; chk_inputs=true)

In [ ]:
solve_wordle("otter")

In [ ]:
solve_wordle("gamma")

In [ ]:
solve_wordle("windy")

In [ ]:
solve_wordle("donut"; chk_inputs=true)

In [ ]:
solve_wordle("child")

In [ ]:
solve_wordle("hobby")

In [ ]:
solve_wordle("wrath")

In [ ]:
solve_wordle("piano")

In [ ]:
solve_wordle("pixel")

#### Run Solver over all Puzzle Words

In [ ]:
words = WORDLE_DF[!, :word]
freqs = WORDLE_DF[!, :freq]

guess_len :: Vector{Int64}   = []
freq_val  :: Vector{Float64} = []
successes :: Vector{Symbol}  = []

i = 0
@time begin
    for word in words
        i += 1
              
        ## Use the default strategy -- pick most frequently used word in filtered Wordle universe.
        res = solve_wordle(word; chk_inputs=true)
        ## res = solve_wordle(word; chk_inputs=false, guess_strategy=freq_letter_strat) ## Don't check input contract and use a strategy to pick next guess.

        ## Check for success or failure.
        if res[3] === :SUCCESS
            push!(guess_len, res[2]  )
            push!(freq_val , freqs[i]) 
            push!(successes, res[3]  )
        else
            println("Failure for puzzle word = $word")
            print(res)
            println("\n")
        end
    end
end

#### Gather Statistics for Solver

In [ ]:
println("Mean guesses          = $(round(mean(guess_len), digits=2))")
println("Weighted mean quesses = $(round(mean(guess_len, weights(freq_val)), digits=2))")

In [ ]:
solve_wordle("folly")

In [ ]:
solve_wordle("cower")